In [7]:
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()
openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [3]:
if 'first-index' not in pc.list_indexes().names():
    pc.create_index(
        name='first-index', 
        dimension=1536, 
        metric='euclidean',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-west-2'
        )
    )

In [4]:
loader = PyPDFLoader("cybersecuirty_sb_factsheets_all.pdf")
pages = loader.load_and_split()

In [5]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
docs = text_splitter.split_documents(pages)


' \ntext_splitter = RecursiveCharacterTextSplitter(\n    chunk_size = 1000,\n    chunk_overlap  = 200,\n    length_function = len,\n)\ndocs = text_splitter.split_documents(pages)\n'

In [4]:

from langchain_openai import OpenAIEmbeddings
model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model = model_name, openai_api_key=os.getenv('OPENAI_API_KEY'))


In [7]:
""" 
embeds = embeddings.embed_documents([sent.page_content for sent in docs])  # Remplacez par vos embeddings
document_ids = [i for i in range(len(embeds))]
"""

' \nembeds = embeddings.embed_documents([sent.page_content for sent in docs])  # Remplacez par vos embeddings\ndocument_ids = [i for i in range(len(embeds))]\n'

In [8]:
""" 
index_name = pc.Index('first-index')
index_name.upsert(vectors=[{"id": str(doc_id), "values": embedding} for doc_id, embedding in zip(document_ids, embeds)])
"""

' \nindex_name = pc.Index(\'first-index\')\nindex_name.upsert(vectors=[{"id": str(doc_id), "values": embedding} for doc_id, embedding in zip(document_ids, embeds)])\n'

In [6]:

index_name = pc.Index('first-index')
text_field = "text"

vectorstore = Pinecone(index_name, embeddings.embed_query,text_field)

In [9]:
query = "How to protect equipment from cyber attacks?"
vectorstore.similarity_search(
    query,  # our search query
    k=2  # return 3 most relevant docs
)

PineconeApiAttributeError: ScoredVector has no attribute 'metadata' at ['['received_data', 'matches', 0]']['metadata']